### chargement et transformation des tables

In [ ]:
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine


aujourdhui = datetime.now()
date_controle = aujourdhui.strftime('%d-%m-%y')


user = 'airflow'
password = 'airflow'
host = 'postgres'
port = '5432'  
database_bronze = 'bronze'
database_silver = 'silver'

db_url_bronze = f"postgresql://{user}:{password}@{host}:{port}/{database_bronze}"
engine_bronze = create_engine(db_url_bronze)

db_url_silver = f"postgresql://{user}:{password}@{host}:{port}/{database_silver}"
engine_silver= create_engine(db_url_silver)

df_clients = pd.read_sql("SELECT * FROM clients", con=engine_bronze)
df_controles = pd.read_sql("SELECT * FROM controles", con=engine_silver)


In [2]:
df_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ID_CLIENT          10000 non-null  object
 1   NOM                9000 non-null   object
 2   PRENOM             9000 non-null   object
 3   CIN                9000 non-null   object
 4   CARTE_SEJOUR       9000 non-null   object
 5   VILLE              9000 non-null   object
 6   NATIONALITE        10000 non-null  object
 7   TELEPHONE          9000 non-null   object
 8   id_intermediaire   10000 non-null  int64 
 9   DATE_CREATION      10000 non-null  object
dtypes: int64(1), object(9)
memory usage: 781.4+ KB


In [206]:
df_clients.head()

,ID_CLIENT,NOM,PRENOM,CIN,CARTE_SEJOUR,VILLE,NATIONALITE,TELEPHONE,id_intermediaire,DATE_CREATION
0,lu1cldwMLx,Hicks,Dawn,SJ109379,OL118027,None,mre,800-778-8075x457,84,2021-05-14
1,k8N4sn4YJb,Wolfe,Kimberly,B136474,O499923,North Alishaport,mre,+1-913-505-2118x425,15,2024-03-27
2,6qzPbzht3H,Burch,Isabel3@,KX010301,HM264290,North Eric`,mre,935-921-5809,13,2021-07-16
3,KZPbHdxc1a,None,Matthew,YT977001,OI901480,South Kenneth,etranger,4332103507,2,2023-02-25
4,3KaFE6Zhq9,Roth=#,None,EB353024,B240907,None,marocain,001-367-758-8791,91,2021-01-03


In [207]:
df_controles.head()

,ID_CONTROLE,DATA,DIM_CONTROLE,DATE_CONTROLE


### table resultat completude de la colonne nom

In [ ]:
df_comp_nom = df_clients[['id_client']].copy()
df_comp_nom['result'] = df_clients['nom'].notnull().astype(int) 

if df_controles.empty:
    df_comp_nom['id_controle'] = 1 
else:
    max_id = df_controles['id_controle'].max()
    df_comp_nom['id_controle'] = max_id + 1 


In [209]:
df_comp_nom.head()

,ID_CLIENT,RESULT,ID_CONTROLE
0,lu1cldwMLx,1,1
1,k8N4sn4YJb,1,1
2,6qzPbzht3H,1,1
3,KZPbHdxc1a,0,1
4,3KaFE6Zhq9,1,1


In [ ]:
df_comp_nom.to_sql('df_comp_nom', con=engine_silver, if_exists='replace', index=False)

### remplissage de la table des controles

In [ ]:
id_controle = df_comp_nom['id_controle'].iloc[0] 
data = "nom"
dim_controle = "completude"

nouvelles_lignes = [[id_controle, data, dim_controle, date_controle]]
nouvelle_ligne_df = pd.DataFrame(nouvelles_lignes, columns=['id_controle', 'data', 'dim_controle', 'date_controle'])
df_controles = pd.concat([df_controles, nouvelle_ligne_df], ignore_index=True)

In [212]:
df_controles.head()

,ID_CONTROLE,DATA,DIM_CONTROLE,DATE_CONTROLE
0,1.0,NOM,COMPLETUDE,23-10-24


### table resultat completude de la colonne pernom

In [ ]:
df_comp_prenom = df_clients[['id_client']].copy()
df_comp_prenom['result'] = df_clients['prenom'].notnull().astype(int) 
max_id = df_controles['id_controle'].max()
df_comp_prenom['id_controle'] = max_id + 1


In [ ]:

df_comp_prenom.to_sql('df_comp_prenom', con=engine_silver, if_exists='replace', index=False)

### remplissage de la table des controles

In [ ]:
id_controle = df_controles['id_controle'].max() + 1
data = "prenom"
dim_controle = "completude"

nouvelles_lignes = [[id_controle, data, dim_controle, date_controle]]
nouvelle_ligne_df = pd.DataFrame(nouvelles_lignes, columns=['id_controle', 'data', 'dim_controle', 'date_controle'])
df_controles = pd.concat([df_controles, nouvelle_ligne_df], ignore_index=True)

### table resultat completude de la colonne cin

In [ ]:
df_comp_cin = df_clients[['id_client']].copy()
df_comp_cin['result'] = df_clients['cin'].notnull().astype(int) 
max_id = df_controles['id_controle'].max()
df_comp_cin['id_controle'] = max_id + 1


In [ ]:

df_comp_cin.to_sql('df_comp_cin', con=engine_silver, if_exists='replace', index=False)

### remplissage de la table des controles

In [ ]:
id_controle = df_controles['id_controle'].max() + 1
data = "cin"
dim_controle = "completude"

nouvelles_lignes = [[id_controle, data, dim_controle, date_controle]]
nouvelle_ligne_df = pd.DataFrame(nouvelles_lignes, columns=['id_controle', 'data', 'dim_controle', 'date_controle'])
df_controles = pd.concat([df_controles, nouvelle_ligne_df], ignore_index=True)

### table resultat completude de la colonne carte sejour

In [ ]:
df_comp_carte_sejour = df_clients[['id_client']].copy()
df_comp_carte_sejour['result'] = df_clients['carte_sejour'].notnull().astype(int) 
max_id = df_controles['id_controle'].max()
df_comp_carte_sejour['id_controle'] = max_id + 1


In [ ]:
df_comp_carte_sejour.to_sql('df_comp_carte_sejour', con=engine_silver, if_exists='replace', index=False)

### remplissage de la table des controles

In [ ]:
id_controle = df_controles['id_controle'].max() + 1
data = "carte_sejour"
dim_controle = "completude"

nouvelles_lignes = [[id_controle, data, dim_controle, date_controle]]
nouvelle_ligne_df = pd.DataFrame(nouvelles_lignes, columns=['id_controle', 'data', 'dim_controle', 'date_controle'])
df_controles = pd.concat([df_controles, nouvelle_ligne_df], ignore_index=True)

### table resultat completude de la colonne ville

In [ ]:
df_comp_ville = df_clients[['id_client']].copy()
df_comp_ville['result'] = df_clients['ville'].notnull().astype(int) 
max_id = df_controles['id_controle'].max()
df_comp_ville['id_controle'] = max_id + 1


In [ ]:
df_comp_ville.to_sql('df_comp_ville', con=engine_silver, if_exists='replace', index=False)


### remplissage de la table des controles

In [ ]:
id_controle = df_controles['id_controle'].max() + 1
data = "ville"
dim_controle = "completude"

nouvelles_lignes = [[id_controle, data, dim_controle, date_controle]]
nouvelle_ligne_df = pd.DataFrame(nouvelles_lignes, columns=['id_controle', 'data', 'dim_controle', 'date_controle'])
df_controles = pd.concat([df_controles, nouvelle_ligne_df], ignore_index=True)

### table resultat completude de la colonne telephone

In [ ]:
df_comp_telephone = df_clients[['id_client']].copy()
df_comp_telephone['result'] = df_clients['telephone'].notnull().astype(int) 
max_id = df_controles['id_controle'].max()
df_comp_telephone['id_controle'] = max_id + 1


In [ ]:

df_comp_telephone.to_sql('df_comp_telephone', con=engine_silver, if_exists='replace', index=False)


### remplissage de la table des controles

In [ ]:
id_controle = df_controles['id_controle'].max() + 1
data = "telephone"
dim_controle = "completude"

nouvelles_lignes = [[id_controle, data, dim_controle, date_controle]]
nouvelle_ligne_df = pd.DataFrame(nouvelles_lignes, columns=['id_controle', 'data', 'dim_controle', 'date_controle'])
df_controles = pd.concat([df_controles, nouvelle_ligne_df], ignore_index=True)

In [ ]:
df_controles.to_sql('controles', con=engine_silver, if_exists='append', index=False)

In [229]:
df_controles.head(20)

,ID_CONTROLE,DATA,DIM_CONTROLE,DATE_CONTROLE
0,1.0,NOM,COMPLETUDE,23-10-24
1,2.0,PRENOM,COMPLETUDE,23-10-24
2,3.0,CIN,COMPLETUDE,23-10-24
3,4.0,CARTE_SEJOUR,COMPLETUDE,23-10-24
4,5.0,VILLE,COMPLETUDE,23-10-24
5,6.0,TELEPHONE,COMPLETUDE,23-10-24
